In [9]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import GroupNormalizer, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.metrics import MAE, RMSE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_forecasting.utils import profile
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse

In [10]:
from math import ceil

In [11]:
from datetime import datetime as dt
from dateutil.relativedelta import *
from scipy import signal
# COMMAND ----------

import time
import pandas as pd
import numpy as np
import gc
import os
from typing import Union
#from tqdm.notebook import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from scipy.sparse import csr_matrix
from scipy import stats
from scipy.special import inv_boxcox
from typing import Tuple

# COMMAND ----------

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.vector_ar.vecm import VECM
from math import sqrt
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [12]:
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import OrdinalEncoder, StandardScaler,MinMaxScaler
from sklearn.impute import SimpleImputer

from datetime import datetime, timedelta
from sklearn.model_selection import GroupKFold

In [13]:
# COMMAND ----------
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold
from sklearn import metrics
from scipy.misc import derivative
from sklearn.model_selection import KFold,TimeSeriesSplit
from sklearn.metrics import roc_auc_score,matthews_corrcoef,make_scorer
#from xgboost import plot_importance
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, roc_auc_score,f1_score, precision_score, recall_score,confusion_matrix,log_loss
from pathlib import Path
from sklearn.metrics import mean_squared_error,mean_squared_log_error,mean_absolute_percentage_error,r2_score
import hyperopt
from hyperopt import hp, tpe, fmin, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
from hyperopt import STATUS_OK

# COMMAND ----------

from pathlib import Path
from sklearn.model_selection import train_test_split

# COMMAND ----------

import datetime


# COMMAND ----------

import os
import shutil

In [14]:
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
import pickle

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.options.mode.chained_assignment = None

## Functions and Classes

In [8]:
def get_start_end_dates(year, week):
    d = dt(year,1,1)
    if(d.weekday()<= 3):
        d = d - timedelta(d.weekday())             
    else:
        d = d + timedelta(7-d.weekday())
    dlt = timedelta(days = (week-1)*7)
    return (d + dlt,  d + dlt + timedelta(days=6))

In [9]:
def get_week_of_month(date):
   first_day = date.replace(day=1)

   day_of_month = date.day

   if(first_day.weekday() == 6):
       adjusted_dom = (1 + first_day.weekday()) / 7
   else:
       adjusted_dom = day_of_month + first_day.weekday()

   return int(ceil(adjusted_dom/7.0))

In [10]:
class TimeBasedCV(object):
    '''
    Parameters 
    ----------
    train_period: int
        number of time units to include in each train set
        default is 30
    test_period: int
        number of time units to include in each test set
        default is 5
    freq: string
        frequency of input parameters. possible values are: days, months, years, weeks, hours, minutes, seconds
        possible values designed to be used by dateutil.relativedelta class
        deafault is days
    '''
    
    
    def __init__(self, train_period=0, test_period=5, freq='week'):
        self.train_period = train_period
        self.test_period = test_period
        self.freq = freq

        
        
    def split(self, data, validation_split_date=None, date_column='ROLLING_WEEK', gap=0):
        '''
        Generate indices to split data into training and test set
        
        Parameters 
        ----------
        data: pandas DataFrame
            your data, contain one column for the record date 
        validation_split_date: datetime.date()
            first date to perform the splitting on.
            if not provided will set to be the minimum date in the data after the first training set
        date_column: string, deafult='record_date'
            date of each record
        gap: int, default=0
            for cases the test set does not come right after the train set,
            *gap* days are left between train and test sets
        
        Returns 
        -------
        train_index ,test_index: 
            list of tuples (train index, test index) similar to sklearn model selection
        '''
        
        # check that date_column exist in the data:
        try:
            data[date_column]
        except:
            raise KeyError(date_column)
                    
        train_indices_list = []
        test_indices_list = []

        #if validation_split_date==None:
        #    validation_split_date = data[date_column].min().date() + eval('relativedelta('+self.freq+'=self.train_period)')
            
        #if validation_split_date==None:
        #    validation_split_period = data[date_column].min() + eval('relativedelta('+self.freq+'=self.train_period)')
        
        MAX_TRAINING_PERIOD=data[date_column].max()
        MIN_TRAINING_PERIOD=data[date_column].min()
        
#         start_training_period = validation_split_date - eval('relativedelta('+self.freq+'=self.train_period)')
#         end_train = start_train + eval('relativedelta('+self.freq+'=self.train_period)')
#         start_test = end_train + eval('relativedelta('+self.freq+'=gap)')
#         end_test = start_test + eval('relativedelta('+self.freq+'=self.test_period)')

        #start_training_period = MIN_TRAINING_PERIOD
        #end_training_period = MAX_TRAINING_PERIOD-(self.test_period*20)
        
        #start_testing_period = end_training_period + 1
        #end_testing_period =  MAX_TRAINING_PERIOD

        for i in range(self.test_period-1,-1,-1):
            
            i=i+1
            
            print('Pass :-',i)
            
            
            # update dates:
            start_training_period = MIN_TRAINING_PERIOD
            end_training_period   = MAX_TRAINING_PERIOD-((i)*10)
            start_testing_period  = end_training_period+1
            end_testing_period    = MAX_TRAINING_PERIOD

            cur_train_indices = list(data[(data[date_column]>=start_training_period) & 
                                     (data[date_column]<=end_training_period)].index)

            # test indices:
            #cur_test_indices = list(data[(data[date_column]>=start_testing_period) &
            #                        (data[date_column]<=end_testing_period)].index)
            
            cur_test_indices = list(data[(data[date_column]>=start_testing_period)].index)
            
            print("Train period:",start_training_period,"-" , end_training_period, ", Test period", start_testing_period, "-", end_testing_period,
                  "# train records", len(cur_train_indices), ", # test records", len(cur_test_indices))

            train_indices_list.append(cur_train_indices)
            test_indices_list.append(cur_test_indices)

            # update dates:
            #start_training_period = MIN_TRAINING_PERIOD
            #end_training_period   = MAX_TRAINING_PERIOD-((i)*20)
            #start_testing_period  = end_training_period+1
            #end_testing_period    = MAX_TRAINING_PERIOD

        # mimic sklearn output  
        index_output = [(train,test) for train,test in zip(train_indices_list,test_indices_list)]
        
        #print(index_output)
        #print(len(index_output))

        self.n_splits = len(index_output)
        
        return index_output
    
    
    def get_n_splits(self):
        """Returns the number of splitting iterations in the cross-validator
        Returns
        -------
        n_splits : int
            Returns the number of splitting iterations in the cross-validator.
        """
        return self.n_splits 

In [11]:
class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)

In [12]:
def rmsle(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y
    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())

In [13]:
def mean_absolute_percentage_error(y_true, y_pred): 
    #y_true, y_pred = check_arrays(y_true, y_pred)

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_pred)) * 100

In [14]:
def plot_feature_imp(model, top_n = 30):
    feature_imp = pd.DataFrame()
    feature_imp['feature'] = model.feature_name_
    feature_imp['importance']  = model.feature_importances_
    feature_imp = feature_imp.sort_values(['importance'], ascending = False)
    feature_imp_disp = feature_imp.head(top_n)
    plt.figure(figsize=(10, 12))
    sns.barplot(x="importance", y="feature", data=feature_imp_disp)
    plt.title('Random Forest Features')
    plt.show() 

In [15]:
def reduce_memory_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        #print(col)
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df.loc[:,col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df.loc[:,col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df.loc[:,col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df.loc[:,col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df.loc[:,col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df.loc[:,col] = df[col].astype(np.float32)
                else:
                    df.loc[:,col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df  

# COMMAND ----------

def joinStrings(stringList):
    return '/'.join(string for string in stringList)

# COMMAND ----------

def unique(a):
    """ return the list with duplicate elements removed """
    return list(set(a))

def intersect(a, b):
    """ return the intersection of two lists """
    return list(set(a) & set(b))

def union(a, b):
    """ return the union of two lists """
    return list(set(a) | set(b))

# COMMAND ----------

def standard_column_names(df_pd):
      """
      Funtion to standardize column names Upper case the names, Replace spaces with underscore
      (,) with blanks,% with PER and - with undescore
      df: Input dataframe
      df_pd: Output with Standardzied column names
      """
      #df_pd=df.toPandas()
      df_pd.columns = df_pd.columns.str.strip().str.upper().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%', 'PER').str.replace('-', '_')
      return df_pd

# COMMAND ----------


# COMMAND ----------

def get_dtypes(df):
  """
  Function to get the categorical and numerical columns of a data frame
  Inputs:
  df: dataframe to get column data types for
  Outputs:
  numerical_cols: list of numerical columns for data frame
  categorical_cols: list of Categorical columns for data frame
  """
  
  numerical_cols=df.select_dtypes([np.int64,np.float64]).columns
  categorical_cols=df.select_dtypes(exclude=[np.number]).columns
  
  return numerical_cols,categorical_cols

# COMMAND ----------

def concat_dim(x: pd.Series) -> pd.Series:
    if x<10:
      return "0"+str(x)
    else:
      return str(x)

# COMMAND ----------

# MAGIC %md #### Time Series Transformations Functions

# COMMAND ----------

# Define power trnnsformations and their inverses
# Square root transformation
def square_root_transformation(x: pd.Series) -> pd.Series:
    return np.sqrt(x)

def square_root_inverse_transformation(x: pd.Series) -> pd.Series:
    return np.square(x)

# Cube root transformation
def cube_root_transformation(x: pd.Series) -> pd.Series:
    return x ** (1 / 3)

def cube_root_inverse_transformation(x: pd.Series) -> pd.Series:
    return x ** 3

# Log transformation
def log_transformation(x: pd.Series) -> pd.Series:
    # Function np.log1p = log(x + 1)
    return np.log1p(x)

def log_inverse_transformation(x: pd.Series) -> pd.Series:
    # Function np.expm1(x) = exp(x) - 1
    return np.expm1(x)

# Box-cox transformation
def box_cox_transformation(x: pd.Series) -> Tuple[np.array, float]:
    x_transformed, lambda_value = stats.boxcox(x)
    return x_transformed, lambda_value
    
def box_cox_inverse_transformation(x: pd.Series, lambda_value: float) -> pd.Series:
    return inv_boxcox(x, lambda_value)

# COMMAND ----------

def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [16]:
def remove_columns(data,remove_cols):
  """
  Function to remove columns from the dataframe
  Inputs:
  data: dataframe to remove columns from
  remove_cols: list of columns to remove
  Outputs:
  data_dropped:Data frame with the columns removed
  """
  data_dropped=data.drop(remove_cols, axis=1)
  return data_dropped

In [17]:
def forecast_accuracy(forecast, actual):
    #mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    #mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    #corr = np.corrcoef(forecast, actual)[0,1]   # corr
    #mins = np.amin(np.hstack([forecast[:,None], 
    #                          actual[:,None]]), axis=1)
    #maxs = np.amax(np.hstack([forecast[:,None], 
    #                          actual[:,None]]), axis=1)
    #minmax = 1 - np.mean(mins/maxs)             # minmax
    return({ 'me':me, 'mae': mae, 
             'rmse':rmse})

In [18]:
def standard_column_names(df_pd):
      """
      Funtion to standardize column names Upper case the names, Replace spaces with underscore
      (,) with blanks,% with PER and - with undescore
      df: Input dataframe
      df_pd: Output with Standardzied column names
      """
      #df_pd=df.toPandas()
      df_pd.columns = df_pd.columns.str.strip().str.upper().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%', 'PER').str.replace('-', '_')
      return df_pd

In [19]:
## Global Variables

In [20]:
NO_OF_WEEKS=60
WINDOW=5
TOP_N=10
FORECAST_HORIZON=13


# COMMAND ----------

week_column='ROLLING_WEEK'
LABEL='SALES_VOLUME'
LEVEL ='PRODUCT_GROUP_CHANNEL_COUNTRY_LE'

# COMMAND ----------

#GLOBAL HYPEROPT PARAMETERS
NUM_EVALS = 25 #number of hyperopt evaluation rounds
N_FOLDS = 5 #number of cross-validation folds on data in each evaluation round

In [21]:
beta=0.5
epsilon=1e-7

## Read Feature Engineeried Data

In [22]:
df_data=pd.read_csv(r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Output\data_with_feature_engineering.csv')

In [23]:
df_data['key']=df_data.apply(lambda x : x['product_id']+x['channel']+x['country'],axis=1)

In [24]:
df_xx=df_data.groupby(['key','product_id', 'channel', 'country'])['sales_volume'].sum().reset_index()

In [25]:
remove_items=list(df_xx[df_xx['sales_volume']==0]['key'].unique())

In [26]:
df_filtered=df_data[~df_data['key'].isin(remove_items)]
df_filtered.shape

(111660, 48)

In [27]:
df_data=df_filtered.copy()

In [28]:
df_data.head()

,product_id,channel,country,week_key,current_price_x,sales_volume,total_stock_volume,store_stock_volume,depot_stock_volume,future_commitment_volume,intake_volume,per_stores_with_stock_count,per_stores_with_sales_count,week,year,start_date,end_date,product_season,product_group,product_subgroup,product_class,product_subclass,original_price,previous_price,current_price_y,key,week_sin365_1,week_cos365_1,week_sin365_2,week_cos365_2,price_change_wrt_previous,price_change_wrt_original,week_of_month,is_month_start,is_month_end,month,is_year_start,is_year_end,is_quarter_start,is_quarter_end,year_2015,year_2016,month_sin365_1,month_cos365_1,percent_change,min_percent,max_cost,min_cost
0,135fc45e,Stores,B,201502,18.98,3.0,39.0,6.0,33.0,0.0,0.0,0.1010,0.0078,2,2015,2015-01-05,2015-01-11,L,26387251,c3567a18,8face522,641d430c,18.98,18.98,16.28,135fc45eStoresB,0.239316,0.970942,0.464723,0.885456,0.0,0.0,2,0,0,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.49
1,135fc45e,Stores,B,201503,18.98,3.0,38.0,6.0,33.0,0.0,-1.0,0.1010,0.0078,3,2015,2015-01-12,2015-01-18,L,26387251,c3567a18,8face522,641d430c,18.98,18.98,16.28,135fc45eStoresB,0.354605,0.935016,0.663123,0.748511,0.0,0.0,3,0,0,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.49
2,135fc45e,Stores,B,201504,18.98,3.0,38.0,6.0,33.0,0.0,-1.0,0.1010,0.0078,4,2015,2015-01-19,2015-01-25,L,26387251,c3567a18,8face522,641d430c,18.98,18.98,16.28,135fc45eStoresB,0.464723,0.885456,0.822984,0.568065,0.0,0.0,4,0,1,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.49
3,135fc45e,Stores,B,201505,18.98,4.0,38.0,6.0,32.0,0.0,0.0,0.1127,0.0117,5,2015,2015-01-26,2015-02-01,L,26387251,c3567a18,8face522,641d430c,18.98,18.98,16.28,135fc45eStoresB,0.568065,0.822984,0.935016,0.354605,0.0,0.0,5,0,1,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.49
4,135fc45e,Stores,B,201506,18.98,4.0,38.0,5.0,32.0,0.0,1.0,0.1088,0.0117,6,2015,2015-02-02,2015-02-08,L,26387251,c3567a18,8face522,641d430c,18.98,18.98,16.28,135fc45eStoresB,0.663123,0.748511,0.992709,0.120537,0.0,0.0,1,1,0,2,1,0,0,0,1,0,0.866025,-0.5,0.0,0.25,14.238281,9.49


In [29]:
df_data['start_date']=pd.to_datetime(df_data['start_date'])
df_data['end_date']=pd.to_datetime(df_data['end_date'])

In [30]:
df_data.shape

(111660, 48)

In [31]:
df_data['rolling_week']=df_data.groupby(['product_id', 'channel', 'country']).week_key.rank(method='dense',ascending=True).astype(int)

In [32]:
df_data['product_group_channel_country']=df_data['product_group']+'-'+df_data['country'].str.upper()+'-'+df_data['channel'].str.upper()

In [33]:
df_data.head()

,product_id,channel,country,week_key,current_price_x,sales_volume,total_stock_volume,store_stock_volume,depot_stock_volume,future_commitment_volume,intake_volume,per_stores_with_stock_count,per_stores_with_sales_count,week,year,start_date,end_date,product_season,product_group,product_subgroup,product_class,product_subclass,original_price,previous_price,current_price_y,key,week_sin365_1,week_cos365_1,week_sin365_2,week_cos365_2,price_change_wrt_previous,price_change_wrt_original,week_of_month,is_month_start,is_month_end,month,is_year_start,is_year_end,is_quarter_start,is_quarter_end,year_2015,year_2016,month_sin365_1,month_cos365_1,percent_change,min_percent,max_cost,min_cost,rolling_week,product_group_channel_country
0,135fc45e,Stores,B,201502,18.98,3.0,39.0,6.0,33.0,0.0,0.0,0.1010,0.0078,2,2015,2015-01-05,2015-01-11,L,26387251,c3567a18,8face522,641d430c,18.98,18.98,16.28,135fc45eStoresB,0.239316,0.970942,0.464723,0.885456,0.0,0.0,2,0,0,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.49,1,26387251-B-STORES
1,135fc45e,Stores,B,201503,18.98,3.0,38.0,6.0,33.0,0.0,-1.0,0.1010,0.0078,3,2015,2015-01-12,2015-01-18,L,26387251,c3567a18,8face522,641d430c,18.98,18.98,16.28,135fc45eStoresB,0.354605,0.935016,0.663123,0.748511,0.0,0.0,3,0,0,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.49,2,26387251-B-STORES
2,135fc45e,Stores,B,201504,18.98,3.0,38.0,6.0,33.0,0.0,-1.0,0.1010,0.0078,4,2015,2015-01-19,2015-01-25,L,26387251,c3567a18,8face522,641d430c,18.98,18.98,16.28,135fc45eStoresB,0.464723,0.885456,0.822984,0.568065,0.0,0.0,4,0,1,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.49,3,26387251-B-STORES
3,135fc45e,Stores,B,201505,18.98,4.0,38.0,6.0,32.0,0.0,0.0,0.1127,0.0117,5,2015,2015-01-26,2015-02-01,L,26387251,c3567a18,8face522,641d430c,18.98,18.98,16.28,135fc45eStoresB,0.568065,0.822984,0.935016,0.354605,0.0,0.0,5,0,1,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.49,4,26387251-B-STORES
4,135fc45e,Stores,B,201506,18.98,4.0,38.0,5.0,32.0,0.0,1.0,0.1088,0.0117,6,2015,2015-02-02,2015-02-08,L,26387251,c3567a18,8face522,641d430c,18.98,18.98,16.28,135fc45eStoresB,0.663123,0.748511,0.992709,0.120537,0.0,0.0,1,1,0,2,1,0,0,0,1,0,0.866025,-0.5,0.0,0.25,14.238281,9.49,5,26387251-B-STORES


## Add Lags,mean encoding and other features

In [34]:
import sys  
sys.path.insert(0, r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Notebooks\fe_utils.py')
%run fe_utils.py

In [35]:
df_data.head()
df_data=standard_column_names(df_data)
df_data.columns

Index(['PRODUCT_ID', 'CHANNEL', 'COUNTRY', 'WEEK_KEY', 'CURRENT_PRICE_X',
       'SALES_VOLUME', 'TOTAL_STOCK_VOLUME', 'STORE_STOCK_VOLUME',
       'DEPOT_STOCK_VOLUME', 'FUTURE_COMMITMENT_VOLUME', 'INTAKE_VOLUME',
       'PER_STORES_WITH_STOCK_COUNT', 'PER_STORES_WITH_SALES_COUNT', 'WEEK',
       'YEAR', 'START_DATE', 'END_DATE', 'PRODUCT_SEASON', 'PRODUCT_GROUP',
       'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS',
       'ORIGINAL_PRICE', 'PREVIOUS_PRICE', 'CURRENT_PRICE_Y', 'KEY',
       'WEEK_SIN365_1', 'WEEK_COS365_1', 'WEEK_SIN365_2', 'WEEK_COS365_2',
       'PRICE_CHANGE_WRT_PREVIOUS', 'PRICE_CHANGE_WRT_ORIGINAL',
       'WEEK_OF_MONTH', 'IS_MONTH_START', 'IS_MONTH_END', 'MONTH',
       'IS_YEAR_START', 'IS_YEAR_END', 'IS_QUARTER_START', 'IS_QUARTER_END',
       'YEAR_2015', 'YEAR_2016', 'MONTH_SIN365_1', 'MONTH_COS365_1',
       'PERCENT_CHANGE', 'MIN_PERCENT', 'MAX_COST', 'MIN_COST', 'ROLLING_WEEK',
       'PRODUCT_GROUP_CHANNEL_COUNTRY'],
      dtype='object')

In [36]:
df_data_fe=fe(df_data)

In [37]:
df_data_fe.shape

(111660, 87)

In [38]:
df_data_fe=reduce_memory_usage(df_data_fe)

Mem. usage decreased to 25.98 Mb (65.1% reduction)


In [39]:
df_data_fe.columns

Index(['PRODUCT_ID', 'CHANNEL', 'COUNTRY', 'WEEK_KEY', 'CURRENT_PRICE_X',
       'SALES_VOLUME', 'TOTAL_STOCK_VOLUME', 'STORE_STOCK_VOLUME',
       'DEPOT_STOCK_VOLUME', 'FUTURE_COMMITMENT_VOLUME', 'INTAKE_VOLUME',
       'PER_STORES_WITH_STOCK_COUNT', 'PER_STORES_WITH_SALES_COUNT', 'WEEK',
       'YEAR', 'START_DATE', 'END_DATE', 'PRODUCT_SEASON', 'PRODUCT_GROUP',
       'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS',
       'ORIGINAL_PRICE', 'PREVIOUS_PRICE', 'CURRENT_PRICE_Y', 'KEY',
       'WEEK_SIN365_1', 'WEEK_COS365_1', 'WEEK_SIN365_2', 'WEEK_COS365_2',
       'PRICE_CHANGE_WRT_PREVIOUS', 'PRICE_CHANGE_WRT_ORIGINAL',
       'WEEK_OF_MONTH', 'IS_MONTH_START', 'IS_MONTH_END', 'MONTH',
       'IS_YEAR_START', 'IS_YEAR_END', 'IS_QUARTER_START', 'IS_QUARTER_END',
       'YEAR_2015', 'YEAR_2016', 'MONTH_SIN365_1', 'MONTH_COS365_1',
       'PERCENT_CHANGE', 'MIN_PERCENT', 'MAX_COST', 'MIN_COST', 'ROLLING_WEEK',
       'PRODUCT_GROUP_CHANNEL_COUNTRY', 'SALES_VOLUME_LAG_1',
    

In [40]:
data_fe=df_data_fe.copy()

In [41]:
data_fe.shape

(111660, 87)

## Begin Training

In [42]:
data_fe.columns

Index(['PRODUCT_ID', 'CHANNEL', 'COUNTRY', 'WEEK_KEY', 'CURRENT_PRICE_X',
       'SALES_VOLUME', 'TOTAL_STOCK_VOLUME', 'STORE_STOCK_VOLUME',
       'DEPOT_STOCK_VOLUME', 'FUTURE_COMMITMENT_VOLUME', 'INTAKE_VOLUME',
       'PER_STORES_WITH_STOCK_COUNT', 'PER_STORES_WITH_SALES_COUNT', 'WEEK',
       'YEAR', 'START_DATE', 'END_DATE', 'PRODUCT_SEASON', 'PRODUCT_GROUP',
       'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS',
       'ORIGINAL_PRICE', 'PREVIOUS_PRICE', 'CURRENT_PRICE_Y', 'KEY',
       'WEEK_SIN365_1', 'WEEK_COS365_1', 'WEEK_SIN365_2', 'WEEK_COS365_2',
       'PRICE_CHANGE_WRT_PREVIOUS', 'PRICE_CHANGE_WRT_ORIGINAL',
       'WEEK_OF_MONTH', 'IS_MONTH_START', 'IS_MONTH_END', 'MONTH',
       'IS_YEAR_START', 'IS_YEAR_END', 'IS_QUARTER_START', 'IS_QUARTER_END',
       'YEAR_2015', 'YEAR_2016', 'MONTH_SIN365_1', 'MONTH_COS365_1',
       'PERCENT_CHANGE', 'MIN_PERCENT', 'MAX_COST', 'MIN_COST', 'ROLLING_WEEK',
       'PRODUCT_GROUP_CHANNEL_COUNTRY', 'SALES_VOLUME_LAG_1',
    

In [43]:
 unwanted_columns=['TOTAL_STOCK_VOLUME', 'STORE_STOCK_VOLUME',
       'DEPOT_STOCK_VOLUME', 'FUTURE_COMMITMENT_VOLUME', 'INTAKE_VOLUME',
       'PER_STORES_WITH_STOCK_COUNT', 'PER_STORES_WITH_SALES_COUNT', 
        'START_DATE', 'END_DATE', 'CURRENT_PRICE_Y', 
       'SALES_VOLUME_PRODUCT_SUBGROUP', 
       'SALES_VOLUME_PRODUCT_GROUP',
       'SALES_VOLUME_PRODUCT_CLASS', 
       'SALES_VOLUME_PRODUCT_SUBCLASS',
       'EXPANDING_MEAN_SALES_VOLUME',
       'EXPANDING_MAX_SALES_VOLUME',
       'EXPANDING_MIN_SALES_VOLUME',
       'MONTH_SIN365_1', 'MONTH_COS365_1',
                  'PRICE_CHANGE_WRT_PREVIOUS',
       'PRICE_CHANGE_WRT_ORIGINAL',
                  'YEAR_2015', 'YEAR_2016','PERCENT_CHANGE', 'MIN_PERCENT', 'MAX_COST', 'MIN_COST']

In [44]:
data_fe.rename(columns={'CURRENT_PRICE_X':'CURRENT_PRICE'},inplace=True)

In [45]:
df_cleaned=remove_columns(data_fe,unwanted_columns)
df_cleaned.shape
#df_cleaned.head()

(111660, 60)

In [46]:
df_cleaned.fillna(0,inplace=True)

In [47]:
cols_to_use_regression=df_cleaned.columns

In [48]:
cat_features_no_one_hot = ['KEY','PRODUCT_ID', 'CHANNEL', 'COUNTRY','PRODUCT_SEASON', 'PRODUCT_GROUP',
       'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS','PRODUCT_GROUP_CHANNEL_COUNTRY']

for c in cat_features_no_one_hot:
    df_cleaned[c] = df_cleaned[c].astype('category')
    

In [49]:
id_cols=[]
not_features = id_cols + list([LABEL])
features = [cols for cols in df_cleaned.columns if cols not in not_features]

In [50]:
num_features=df_cleaned.columns[df_cleaned.dtypes!='category']
char_features=df_cleaned.columns[df_cleaned.dtypes=='category']

In [51]:
num_features = num_features.drop(['ROLLING_WEEK',
                                 'IS_MONTH_START','IS_MONTH_END','IS_YEAR_START','IS_YEAR_END',
                                  'IS_QUARTER_START','IS_QUARTER_END',
                                  'WEEK_SIN365_1','WEEK_COS365_1', 'WEEK_SIN365_2', 'WEEK_COS365_2'])

In [52]:
# encode numerical features 
for i, feature in enumerate(num_features):
    df_cleaned[feature].fillna(df_cleaned[feature].median(), inplace = True)

In [53]:
LAST_WEEK=df_cleaned[week_column].max()
FIRST_WEEK=LAST_WEEK-NO_OF_WEEKS
if FIRST_WEEK > 0:
  FIRST_WEEK=FIRST_WEEK
else:
  FIRST_WEEK=0
print(FIRST_WEEK)
print(LAST_WEEK)

0
60


In [54]:
LAST_TRAINING_WEEK=df_cleaned[week_column].max()
FIRST_TRAINING_WEEK=FIRST_WEEK
print(FIRST_TRAINING_WEEK)
print(LAST_TRAINING_WEEK)
print(f'We have {(LAST_TRAINING_WEEK - FIRST_TRAINING_WEEK+1)} week of training history')

0
60
We have 61 week of training history


In [55]:
# Creating a copy of the data for modelling
mod_data = df_cleaned.copy(deep = True)

# Dropping the NaN rows which would be there in the data due to the lags
mod_data=mod_data[(mod_data['ROLLING_WEEK']>=FIRST_TRAINING_WEEK)&(mod_data['ROLLING_WEEK']<=LAST_TRAINING_WEEK)]
data=mod_data[cols_to_use_regression]
data.shape

(111660, 60)

In [56]:
TRAINING_WEEKS=data['ROLLING_WEEK'].max()-WINDOW*2
VALIDATION_WEEKS=TRAINING_WEEKS+WINDOW
TESTING_WEEKS=TRAINING_WEEKS+WINDOW*2
print('Total Training WEEKS FROM 0 - ',TRAINING_WEEKS)
print('Total Validation WEEKS FROM',TRAINING_WEEKS,' - ',VALIDATION_WEEKS)
print('Total Validation WEEKS FROM',VALIDATION_WEEKS,' - ',TESTING_WEEKS)

Total Training WEEKS FROM 0 -  50
Total Validation WEEKS FROM 50  -  55
Total Validation WEEKS FROM 55  -  60


In [57]:
data=reduce_memory_usage(data)

Mem. usage decreased to 12.05 Mb (0.0% reduction)


In [58]:
data.to_csv(r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Output\sales_volume_engineered.csv',index=False)

### Build Label ENCODINGS for Categorical variables

In [59]:
cat_features=char_features
num_features=num_features
cat_features_LE=cat_features+'_LE'
cat_features_LE

Index(['PRODUCT_ID_LE', 'CHANNEL_LE', 'COUNTRY_LE', 'PRODUCT_SEASON_LE',
       'PRODUCT_GROUP_LE', 'PRODUCT_SUBGROUP_LE', 'PRODUCT_CLASS_LE',
       'PRODUCT_SUBCLASS_LE', 'KEY_LE', 'PRODUCT_GROUP_CHANNEL_COUNTRY_LE'],
      dtype='object')

In [60]:
data_embedding=data.copy()

for index,value in enumerate(cat_features):
    new_col=value+'_LE'
    globals()[f'label_encoder_{value}'] = LabelEncoderExt()
    globals()[f'label_encoder_{value}'].fit(data_embedding[value])
    #print(globals()[f'label_encoder_{value}'].classes_)
    data_embedding[new_col]=globals()[f'label_encoder_{value}'].transform(data_embedding[value])
    data_embedding.drop(value,axis=1, inplace=True)
    data_embedding[new_col]=data_embedding[new_col].astype('float')
    globals()[f'le_dict_{value}'] = dict(zip(globals()[f'label_encoder_{value}'].classes_, globals()[f'label_encoder_{value}'].transform(globals()[f'label_encoder_{value}'].classes_)))
    val='dict_'+value
    val_LE='LE_'+value
    with open('C:/Users/kishor.kukreja/OneDrive - Fractal Analytics Pvt. Ltd/Desktop/WIP/Tech Challenge/Encoding/'+val, 'wb') as f:
        pickle.dump(globals()[f'le_dict_{value}'], f)
    with open('C:/Users/kishor.kukreja/OneDrive - Fractal Analytics Pvt. Ltd/Desktop/WIP/Tech Challenge/Encoding/'+val_LE, 'wb') as f:
        pickle.dump(globals()[f'label_encoder_{value}'], f)

### Hyper Parameter Tuning Random Forest per Product group - Channel- country-- total 19 models

In [61]:
data.PRODUCT_GROUP_CHANNEL_COUNTRY.unique()

['26387251-B-STORES', 'bca94c97-A-STORES', 'bca94c97-B-STORES', 'bca94c97-B-ONLINE', 'edf80f3a-B-STORES', ..., 'c8e7b2df-A-STORES', '49e9a58f-B-ONLINE', '3bc0de94-B-ONLINE', '49e9a58f-B-STORES', '3bc0de94-B-STORES']
Length: 19
Categories (19, object): ['26387251-A-STORES', '26387251-B-ONLINE', '26387251-B-STORES', '3bc0de94-B-ONLINE', ..., 'c8e7b2df-B-STORES', 'edf80f3a-A-STORES', 'edf80f3a-B-ONLINE', 'edf80f3a-B-STORES']

In [62]:
data_embedding.head()

,WEEK_KEY,CURRENT_PRICE,SALES_VOLUME,WEEK,YEAR,ORIGINAL_PRICE,PREVIOUS_PRICE,WEEK_SIN365_1,WEEK_COS365_1,WEEK_SIN365_2,WEEK_COS365_2,WEEK_OF_MONTH,IS_MONTH_START,IS_MONTH_END,MONTH,IS_YEAR_START,IS_YEAR_END,IS_QUARTER_START,IS_QUARTER_END,ROLLING_WEEK,SALES_VOLUME_LAG_1,SALES_VOLUME_LAG_2,SALES_VOLUME_LAG_3,SALES_VOLUME_LAG_4,SALES_VOLUME_LAG_5,SALES_VOLUME_LAG_6,SALES_VOLUME_LAG_7,SALES_VOLUME_LAG_8,SALES_VOLUME_LAG_9,SALES_VOLUME_LAG_10,SALES_VOLUME_LAG_11,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_1,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_2,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_3,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_4,SALES_VOLUME_PRODUCT_GROUP_LAG_1,SALES_VOLUME_PRODUCT_GROUP_LAG_2,SALES_VOLUME_PRODUCT_GROUP_LAG_3,SALES_VOLUME_PRODUCT_GROUP_LAG_4,SALES_VOLUME_PRODUCT_CLASS_LAG_1,SALES_VOLUME_PRODUCT_CLASS_LAG_2,SALES_VOLUME_PRODUCT_CLASS_LAG_3,SALES_VOLUME_PRODUCT_CLASS_LAG_4,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_1,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_2,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_3,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_4,EXPANDING_MEAN_SALES_VOLUME_4WEEK,EXPANDING_MEAN_SALES_VOLUME_12WEEK,EXPANDING_MEAN_SALES_VOLUME_8WEEK,PRODUCT_ID_LE,CHANNEL_LE,COUNTRY_LE,PRODUCT_SEASON_LE,PRODUCT_GROUP_LE,PRODUCT_SUBGROUP_LE,PRODUCT_CLASS_LE,PRODUCT_SUBCLASS_LE,KEY_LE,PRODUCT_GROUP_CHANNEL_COUNTRY_LE
0,201502,18.984375,3.0,2,2015,18.984375,18.984375,0.239258,0.970703,0.464844,0.885254,2,0,0,1,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,15.0,1.0,1.0,0.0,0.0,9.0,25.0,30.0,29.0,2.0
1,201503,18.984375,3.0,3,2015,18.984375,18.984375,0.354492,0.935059,0.663086,0.748535,3,0,0,1,0,0,0,0,2,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,15.0,1.0,1.0,0.0,0.0,9.0,25.0,30.0,29.0,2.0
2,201504,18.984375,3.0,4,2015,18.984375,18.984375,0.464844,0.885254,0.822754,0.567871,4,0,1,1,0,0,0,0,3,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.236572,0.000000,0.000000,0.0,0.408447,0.000000,0.000000,0.0,0.191772,0.000000,0.000000,0.0,0.392822,0.000000,0.000000,0.0,0.000000,0.0,0.0,15.0,1.0,1.0,0.0,0.0,9.0,25.0,30.0,29.0,2.0
3,201505,18.984375,4.0,5,2015,18.984375,18.984375,0.567871,0.822754,0.935059,0.354492,5,0,1,1,0,0,0,0,4,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.699707,0.236572,0.000000,0.0,1.066406,0.408447,0.000000,0.0,0.246582,0.191772,0.000000,0.0,0.571289,0.392822,0.000000,0.0,0.000000,0.0,0.0,15.0,1.0,1.0,0.0,0.0,9.0,25.0,30.0,29.0,2.0
4,201506,18.984375,4.0,6,2015,18.984375,18.984375,0.663086,0.748535,0.992676,0.120544,1,1,0,2,1,0,0,0,5,4.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.128906,0.699707,0.236572,0.0,1.798828,1.066406,0.408447,0.0,0.328857,0.246582,0.191772,0.0,0.678711,0.571289,0.392822,0.0,3.400391,0.0,0.0,15.0,1.0,1.0,0.0,0.0,9.0,25.0,30.0,29.0,2.0


In [64]:
model_data=data_embedding.copy()
model_features=model_data.columns.to_list()
#print(model_features)
model_features.remove(LABEL)
model_label=LABEL
cols_not_to_consider_modelling=['WEEK_KEY','YEAR','ROLLING_WEEK']
cols_not_to_consider=['PRODUCT_ID_LE']

In [65]:
## Scaling Data

In [66]:

train_data = model_data[model_data['ROLLING_WEEK'] <= TRAINING_WEEKS]

for index,value in enumerate(num_features):
    new_col=value+''
    globals()[f'min_max_scalar_{value}'] = MinMaxScaler()
    globals()[f'min_max_scalar_{value}'].fit(np.array(train_data[value]).reshape(-1,1))
    
    train_data[new_col]=globals()[f'min_max_scalar_{value}'].transform(np.array(train_data[value]).reshape(-1,1))
    
    train_data[new_col]=train_data[new_col].astype('float')
    
    val_SS=value
    with open(r'C:/Users/kishor.kukreja/OneDrive - Fractal Analytics Pvt. Ltd/Desktop/WIP/Tech Challenge/Scaling/'+val_SS, 'wb') as f:
        pickle.dump(globals()[f'min_max_scalar_{value}'], f)

x_train = train_data[model_features]
y_train = train_data[model_label]


val_data = model_data[(model_data['ROLLING_WEEK'] >TRAINING_WEEKS)&(model_data['ROLLING_WEEK'] <= VALIDATION_WEEKS)]

for index,value in enumerate(num_features):
    new_col=value+''
    val_data[new_col]=globals()[f'min_max_scalar_{value}'].transform(np.array(val_data[value]).reshape(-1,1))
    val_data[new_col]=val_data[new_col].astype('float')

x_val = val_data[model_features]
y_val = val_data[model_label]


test_data = model_data[(model_data['ROLLING_WEEK'] >VALIDATION_WEEKS)&(model_data['ROLLING_WEEK'] <= TESTING_WEEKS)]
for index,value in enumerate(num_features):
    new_col=value+''
    test_data[new_col]=globals()[f'min_max_scalar_{value}'].transform(np.array(test_data[value]).reshape(-1,1))
    test_data[new_col]=test_data[new_col].astype('float')

x_test = test_data[model_features]
y_test = test_data[model_label]

# Train + validation set
train_val_data = model_data[model_data['ROLLING_WEEK'] <= VALIDATION_WEEKS]
for index,value in enumerate(num_features):
    new_col=value+''
    train_val_data[new_col]=globals()[f'min_max_scalar_{value}'].transform(np.array(train_val_data[value]).reshape(-1,1))
    train_val_data[new_col]=train_val_data[new_col].astype('float')
    
x_train_val = train_val_data[model_features]
y_train_val = train_val_data[model_label]

# COMMAND ----------

print('Training data Shape')
print(x_train.shape)
print(y_train.shape)
print('Validation data Shape')
print(x_val.shape)
print(y_val.shape)
print('Testing data Shape')
print(x_test.shape)
print(y_test.shape)
print('Training+Validation data Shape')
print(x_train_val.shape)
print(y_train_val.shape)

Training data Shape
(93050, 59)
(93050,)
Validation data Shape
(9305, 59)
(9305,)
Testing data Shape
(9305, 59)
(9305,)
Training+Validation data Shape
(102355, 59)
(102355,)


### Bayesian Optimization

In [67]:
out_rmse = []
out_fb = []
out_tsb_v =[]
out_params =[]
out_subclass=[]


def quick_hyperopt_ts(train_data, train_labels,val_data,val_labels,test_data,test_labels,n_folds, num_evals, value,diagnostic=False):
    
    ###-------------------------------------- OBJECTIVE FUNCTION ----------------###
    
    
    UNIQUE_SKU = data_embedding[data_embedding['PRODUCT_GROUP_CHANNEL_COUNTRY_LE']==value]['KEY_LE'].unique()
    
    integer_params = ['max_depth',
                      'n_estimators',
                      'min_samples_leaf',
                      'min_samples_split']
    
    def objective_function(params):
        #reg = RandomForestRegressor(**params)
        

        for param in integer_params:
            params[param] = int(params[param])
        
        
        X_tr=train_data.drop(columns=cols_not_to_consider_modelling)
        y_tr=train_labels
        X_te=val_data.reset_index(drop=True).drop(columns=cols_not_to_consider_modelling)
        y_te=val_labels.reset_index(drop=True)
        
        print(X_tr.shape)
        
        #X_DEMAND_val = val_data[cols_not_to_consider]


        model_rfe = RandomForestRegressor(**params,random_state=42,n_jobs=-1)
        model_rfe.fit(X_tr, y_tr)
        predictions = model_rfe.predict(X_te)

        #forecast=forecast_accuracy(predictions,y_te)
        #print('Forecast ACCURACY:----------')
        #print(forecast)
        print(params)

         # predict oof
        """def pred_func(x):
            preds_idx = model_rfe.predict(x)
            print(preds_idx)
            oof_tsb = np.sum(y_te-preds_idx)/(np.sum(np.abs(y_te-preds_idx))/len(preds_idx))
            return oof_tsb
        
        a= X_te.groupby("KEY_LE").agg(pred_func)"""
        #----------- Lists to store TSB/COH/RMSE
        
        tsb=[]
        tsb_flag = []
        
        for i in UNIQUE_SKU:
            
            X_te2 = X_te[X_te.KEY_LE==i]
            
            
            preds_idx = model_rfe.predict(X_te2)
            y_te2 = y_te[np.min(X_te2.index):np.max(X_te2.index)+1]
            #X_DEMAND_val2 = X_DEMAND_val[np.min(X_te2.index):np.max(X_te2.index)+1]
            #print(np.min(X_te2.index))
            #print(np.max(X_te2.index))
            
            
            #--------------TSB--------------------
            #print(preds_idx.shape)
            #print(y_te2.shape)
            oof_tsb = np.sum(y_te2-preds_idx)/(np.sum(np.abs(y_te2-preds_idx))/len(preds_idx))
            tsb.append(oof_tsb)
            g=np.where((oof_tsb>=-3.75)&(oof_tsb<=3.75),0,1)
            tsb_flag.append(g)
            
           
        print('tsb_flag_violations:',np.nansum(tsb_flag),"/",len(tsb_flag))
        
        preds_idx = model_rfe.predict(X_te)
        oof_rmse = np.sqrt(metrics.mean_squared_error(y_te, preds_idx))
        print(f'RMSE::{oof_rmse}')
        

        #------------------------------------- Scoring Metrics----------
        
        beta_squared = beta**2
        no_of_violations=np.nansum(tsb_flag)
        rmse=oof_rmse
            
        fb = (1+beta_squared)*no_of_violations*rmse / (beta_squared*no_of_violations + rmse + epsilon)
        print(f'Final loss is {fb:.10f}')

        #print(f'aaaaa {model_rf}')
        out_rmse.append(rmse)
        out_fb.append(fb)
        out_tsb_v.append(str(np.nansum(tsb_flag))+"/"+str(len(tsb_flag)))
        out_params.append(str(model_rfe))
        out_subclass.append(str(value))
        
        score=fb

        return {'loss': score, 'status': STATUS_OK}
    
     ###-------------------------------------- PARAMETER SPACE ----------------###
    
    space= {

    'max_depth': hp.quniform('max_depth', 10, 100, 10),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 20, 1),
    'min_samples_split': hp.quniform('min_samples_split', 5, 100, 5),
    'n_estimators': hp.quniform('n_estimators', 600, 700, 50)
    #'n_estimators': 700

    }
    
    
    ###-------------------------------------- RUN OPTIMIZIZATION ----------------###
    
    tpe_algorithm = tpe.suggest
    
    trials = Trials()
    best = fmin(fn=objective_function,
                space=space,
                algo=tpe_algorithm,
                max_evals=NUM_EVALS, 
                trials=trials)
    
    #cast floats of integer params to int
    for param in integer_params:
        best[param] = int(best[param])
    
    print('{' + '\n'.join('{}: {}'.format(k, v) for k, v in best.items()) + '}')
    
    if diagnostic:
        return(best, trials)
    else:
        return(best)

In [68]:
def filter_data(model_data,value,cols_not_to_consider):    
    
    data=model_data[model_data[LEVEL]==value]
    print(data.shape)

    # Training set 
    train_data = data[data['ROLLING_WEEK'] <= TRAINING_WEEKS]
    x_train = train_data[model_features]
    y_train = train_data[model_label]
    #x_train.drop(columns=cols_not_to_consider,inplace=True)

    # Validation set 
    val_data = data[(data['ROLLING_WEEK'] >TRAINING_WEEKS)&(data['ROLLING_WEEK'] <= VALIDATION_WEEKS)]
    x_val = val_data[model_features]
    y_val = val_data[model_label]
    #x_val.drop(columns=cols_not_to_consider,inplace=True)

    # Test set of 
    test_data = data[(data['ROLLING_WEEK'] >VALIDATION_WEEKS)&(data['ROLLING_WEEK'] <= TESTING_WEEKS)]
    x_test = test_data[model_features]
    y_test = test_data[model_label]
    #x_test.drop(columns=cols_not_to_consider,inplace=True)
    
    # Train + validation set
    train_val_data = data[data['ROLLING_WEEK'] <= VALIDATION_WEEKS]
    x_train_val = train_val_data[model_features]
    y_train_val = train_val_data[model_label]
    #x_train_val.drop(columns=cols_not_to_consider,inplace=True)
    
    print('Training data Shape')
    print(x_train.shape)
    print(y_train.shape)
    print('Validation data Shape')
    print(x_val.shape)
    print(y_val.shape)
    print('Testing data Shape')
    print(x_test.shape)
    print(y_test.shape)
    print('Training+Validation data Shape')
    print(x_train_val.shape)
    print(y_train_val.shape)
    
    return x_train,y_train,x_val,y_val,x_test,y_test,x_train_val,y_train_val

def filter_data2(model_data,value,cols_not_to_consider):    
    data=model_data[model_data[LEVEL]==value]

    
    train_data = data[data['ROLLING_WEEK'] < TRAINING_WEEKS]
    x_train = train_data[model_features]
    y_train = train_data[model_label]
    x_train.drop(columns=cols_not_to_consider,inplace=True)

    
    val_data = data[(data['ROLLING_WEEK'] >TRAINING_WEEKS)&(data['ROLLING_WEEK'] <= VALIDATION_WEEKS)]
    x_val = val_data[model_features]
    y_val = val_data[model_label]
    x_val.drop(columns=cols_not_to_consider,inplace=True)

    
    test_data = data[(data['ROLLING_WEEK'] >VALIDATION_WEEKS)&(data['ROLLING_WEEK'] <= TESTING_WEEKS)]
    x_test = test_data[model_features]
    y_test = test_data[model_label]
    x_test.drop(columns=cols_not_to_consider,inplace=True)
    
    # Train + validation set
    train_val_data = data[data['ROLLING_WEEK'] <= VALIDATION_WEEKS]
    x_train_val = train_val_data[model_features]
    y_train_val = train_val_data[model_label]
    x_train_val.drop(columns=cols_not_to_consider,inplace=True)
    
    print('Training data Shape')
    print(x_train.shape)
    print(y_train.shape)
    print('Validation data Shape')
    print(x_val.shape)
    print(y_val.shape)
    print('Testing data Shape')
    print(x_test.shape)
    print(y_test.shape)
    print('Training+Validation data Shape')
    print(x_train_val.shape)
    print(y_train_val.shape)
    
    return x_train,y_train,x_val,y_val,x_test,y_test,x_train_val,y_train_val




#--------------------- Scaled-------------------
def filter_data_scaled(model_data,value,cols_not_to_consider):    
    
    data=model_data[model_data[LEVEL]==value]

    
    train_data = data[data['ROLLING_WEEK'] <= TRAINING_WEEKS]
    
    #----------- Scaling
    for index,column in enumerate(num_features):
        new_col=column+''
        train_data[new_col]=globals()[f'min_max_scalar_{column}'].transform(np.array(train_data[column]).reshape(-1,1))
    
    x_train = train_data[model_features]
    y_train = train_data[model_label]
    x_train.drop(columns=cols_not_to_consider,inplace=True)

    
    
    
    val_data = data[(data['ROLLING_WEEK'] >TRAINING_WEEKS)&(data['ROLLING_WEEK'] <= VALIDATION_WEEKS)]

        #----------- Scaling
    for index,column in enumerate(num_features):
        new_col=column+''
        val_data[new_col]=globals()[f'min_max_scalar_{column}'].transform(np.array(val_data[column]).reshape(-1,1))
        
        
    x_val = val_data[model_features]
    y_val = val_data[model_label]
    x_val.drop(columns=cols_not_to_consider,inplace=True)

    
    test_data = data[(data['ROLLING_WEEK'] >VALIDATION_WEEKS)&(data['ROLLING_WEEK'] <= TESTING_WEEKS)]
    
    #----------- Scaling
    for index,column in enumerate(num_features):
        new_col=column+''
        test_data[new_col]=globals()[f'min_max_scalar_{column}'].transform(np.array(test_data[column]).reshape(-1,1))
    
    x_test = test_data[model_features]
    y_test = test_data[model_label]
    x_test.drop(columns=cols_not_to_consider,inplace=True)
    
    # Train + validation set
    train_val_data = data[data['ROLLING_WEEK'] <= VALIDATION_WEEKS]
    
        #----------- Scaling
    for index,column in enumerate(num_features):
        new_col=column+''
        train_val_data[new_col]=globals()[f'min_max_scalar_{column}'].transform(np.array(train_val_data[column]).reshape(-1,1))
        
    x_train_val = train_val_data[model_features]
    y_train_val = train_val_data[model_label]
    x_train_val.drop(columns=cols_not_to_consider,inplace=True)
    
    print('Training data Shape')
    print(x_train.shape)
    print(y_train.shape)
    print('Validation data Shape')
    print(x_val.shape)
    print(y_val.shape)
    print('Testing data Shape')
    print(x_test.shape)
    print(y_test.shape)
    print('Training+Validation data Shape')
    print(x_train_val.shape)
    print(y_train_val.shape)
    
    return x_train,y_train,x_val,y_val,x_test,y_test,x_train_val,y_train_val

In [69]:
model_data[LEVEL].unique()

array([ 2., 11., 13., 12., 19., 17.,  9.,  7.,  8.,  0.,  1., 16., 18.,
       15., 14.,  5.,  3.,  6.,  4.])

In [70]:
UNIQUE_KEY=model_data[LEVEL].unique()

In [71]:
#obtain optimised parameter dictionary
#RUN FOR 1 SKU PLANT COMBINATION ## NO Time Series Cross validation
def run_rfe_hyperparameter_onesku(model_data,value,cols_not_to_consider):
    x_train,y_train,x_val,y_val,x_test,y_test,x_train_val,y_train_val=filter_data(model_data,value,cols_not_to_consider)
    
    print(x_train_val.shape)
    rfe_params = quick_hyperopt_ts(x_train,y_train,x_val,y_val,x_test,y_test,N_FOLDS, NUM_EVALS,value)
    return rfe_params

## Model Per subclass

In [72]:
count =0
for key,value in enumerate(UNIQUE_KEY):
    count = count+1
    print('Product Country Channel Count-->',count)
    print('Product Country Channel LE--> ',"--",value)
    
    
    globals()[f'best_rfe_params_{value}'] =run_rfe_hyperparameter_onesku(model_data,value,cols_not_to_consider)
    val='BEST_PARAMS_'+str(value)
    
    print(globals()[f'best_rfe_params_{value}'])
    
    
    with open(r'C:/Users/kishor.kukreja/OneDrive - Fractal Analytics Pvt. Ltd/Desktop/WIP/Tech Challenge/BestParams/'+str(val), 'wb') as f:
        pickle.dump(globals()[f'best_rfe_params_{value}'], f)

Product Country Channel Count--> 1
Product Country Channel LE-->  -- 2.0
(16140, 60)
Training data Shape
(13450, 59)
(13450,)
Validation data Shape
(1345, 59)
(1345,)
Testing data Shape
(1345, 59)
(1345,)
Training+Validation data Shape
(14795, 59)
(14795,)
(14795, 59)
(13450, 56)                                                                                                            
{'max_depth': 10, 'min_samples_leaf': 11, 'min_samples_split': 75, 'n_estimators': 650}                                
tsb_flag_violations:                                                                                                   
190                                                                                                                    
/                                                                                                                      
269                                                                                                                    
RMSE::28.73

RMSE::28.58844576967222                                                                                                
Final loss is 90.2733536555                                                                                            
(13450, 56)                                                                                                            
{'max_depth': 30, 'min_samples_leaf': 3, 'min_samples_split': 30, 'n_estimators': 600}                                 
tsb_flag_violations:                                                                                                   
195                                                                                                                    
/                                                                                                                      
269                                                                                                                    
RMSE::28.847326706458563                

(7350, 56)                                                                                                             
{'max_depth': 90, 'min_samples_leaf': 8, 'min_samples_split': 25, 'n_estimators': 650}                                 
tsb_flag_violations:                                                                                                   
100                                                                                                                    
/                                                                                                                      
147                                                                                                                    
RMSE::1.5699275124486411                                                                                               
Final loss is 7.3858288934                                                                                             
(7350, 56)                              

(7350, 56)                                                                                                             
{'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 35, 'n_estimators': 650}                                 
tsb_flag_violations:                                                                                                   
100                                                                                                                    
/                                                                                                                      
147                                                                                                                    
RMSE::1.5750990396719684                                                                                               
Final loss is 7.4087166683                                                                                             
(7350, 56)                              

(9900, 56)                                                                                                             
{'max_depth': 40, 'min_samples_leaf': 18, 'min_samples_split': 55, 'n_estimators': 650}                                
tsb_flag_violations:                                                                                                   
122                                                                                                                    
/                                                                                                                      
198                                                                                                                    
RMSE::13.283108671342855                                                                                               
Final loss is 46.2661087626                                                                                            
(9900, 56)                              

...

(400, 56)                                                                                                              
{'max_depth': 90, 'min_samples_leaf': 20, 'min_samples_split': 55, 'n_estimators': 650}                                
tsb_flag_violations:                                                                                                   
8                                                                                                                      
/                                                                                                                      
8                                                                                                                      
RMSE::1.2814137435253237                                                                                               
Final loss is 3.9050659400                                                                                             
(400, 56)                               

(50, 56)                                                                                                               
{'max_depth': 70, 'min_samples_leaf': 7, 'min_samples_split': 30, 'n_estimators': 650}                                 
tsb_flag_violations:                                                                                                   
1                                                                                                                      
/                                                                                                                      
1                                                                                                                      
RMSE::1.0954451150103321                                                                                               
Final loss is 1.0177347829                                                                                             
(50, 56)                                

(50, 56)                                                                                                               
{'max_depth': 30, 'min_samples_leaf': 18, 'min_samples_split': 90, 'n_estimators': 600}                                
tsb_flag_violations:                                                                                                   
1                                                                                                                      
/                                                                                                                      
1                                                                                                                      
RMSE::1.0954451150103321                                                                                               
Final loss is 1.0177347829                                                                                             
(50, 56)                                

(100, 56)                                                                                                              
{'max_depth': 70, 'min_samples_leaf': 4, 'min_samples_split': 40, 'n_estimators': 650}                                 
tsb_flag_violations:                                                                                                   
2                                                                                                                      
/                                                                                                                      
2                                                                                                                      
RMSE::0.0                                                                                                              
Final loss is 0.0000000000                                                                                             
(100, 56)                               

(100, 56)                                                                                                              
{'max_depth': 80, 'min_samples_leaf': 5, 'min_samples_split': 65, 'n_estimators': 650}                                 
tsb_flag_violations:                                                                                                   
2                                                                                                                      
/                                                                                                                      
2                                                                                                                      
RMSE::0.0                                                                                                              
Final loss is 0.0000000000                                                                                             
(100, 56)                               

In [73]:
out_df = pd.DataFrame({'TSB Flag Violations':out_tsb_v,
                       'RMSE':out_rmse,
                       'Parameter':out_params,
                       'Final Score':out_fb,
                      'Product Sub Class LE':out_subclass})
#out_df.to_csv('ML_1UoM_RFparams_20211211.csv')
out_df

,TSB Flag Violations,RMSE,Parameter,Final Score,Product Sub Class LE
0,190/269,28.738552,"RandomForestRegressor(max_depth=10, min_sample...",89.526964,2.0
1,194/269,28.582458,"RandomForestRegressor(max_depth=10, min_sample...",89.919890,2.0
2,193/269,28.557376,"RandomForestRegressor(max_depth=30, min_sample...",89.697987,2.0
3,194/269,28.721865,"RandomForestRegressor(max_depth=50, min_sample...",90.195339,2.0
4,192/269,28.570620,"RandomForestRegressor(max_depth=30, min_sample...",89.550651,2.0
5,193/269,28.351172,"RandomForestRegressor(max_depth=50, min_sample...",89.290020,2.0
6,194/269,29.306715,"RandomForestRegressor(max_depth=80, min_sample...",91.340168,2.0
7,187/269,28.848902,"RandomForestRegressor(max_depth=10, min_sample...",89.200116,2.0
8,195/269,29.076403,"RandomForestRegressor(max_depth=100, min_sampl...",91.066437,2.0
9,196/269,28.859112,"RandomForestRegressor(max_depth=40, min_sample...",90.811238,2.0


## Fitting Best Model Random Forest PER combination

In [74]:
def fit_best_model_onesku_tscv(model_data,value,cols_not_to_consider,cols_not_to_consider_modelling,n_folds,model_rfe):
    
    x_train,y_train,x_val,y_val,x_test,y_test,x_train_val,y_train_val=filter_data_scaled(model_data,value,cols_not_to_consider)
    
    #oof_preds_df = pd.DataFrame(index=x_train_val.index)##.to_numpy().flatten()
    #oof_preds=oof_preds_df.copy()
    #oof_preds['predictions']=0
    
    print(cols_not_to_consider_modelling)
    preds=  np.zeros(x_test.shape[0])
    
#    x_train_val=x_train_val.drop(columns=cols_not_to_consider_modelling)
#    x_test=x_test.drop(columns=cols_not_to_consider_modelling)
    
    print(x_train_val.shape)
    tscv = TimeBasedCV(train_period=0,
                       test_period=n_folds,
                       freq='period')

    for i,(train_index, test_index) in enumerate(tscv.split(x_train_val)):
                
            #print(i)
                #print(len(train_index))
                #print(len(test_index))
                #print(train_index)
                try:
                    x_train_val=x_train_val.drop(columns=cols_not_to_consider_modelling)
                except:
                    x_train_val=x_train_val
                    
                X_tr=np.array(x_train_val.loc[train_index,:])
                y_tr=np.array(y_train_val.loc[train_index])
                X_te=np.array(x_train_val.loc[test_index,:])#[features]
                y_te=np.array(y_train_val.loc[test_index])
                
                
                
#                 tmp=x_train_val.copy()
#---------------------- Remove unwanted columns
                #X_tr=np.array(X_tr.drop(columns= cols_not_to_consider))
                #print(len(X_tr.columns))
                #X_te=np.array(X_te.drop(columns= cols_not_to_consider))
                
#                 y_tr=np.array(tmp.loc[train_index])
#                 X_te=np.array(tmp.loc[test_index,:])#[features]
#                 y_te=np.array(tmp.loc[test_index])

#                X_tr=X_tr.drop(columns=cols_not_to_consider_modelling)
#                X_te=X_te.drop(columns=cols_not_to_consider_modelling)
                
                model_rfe.fit(X_tr, y_tr)
                predictions = model_rfe.predict(X_te)

                forecast=forecast_accuracy(predictions,y_te)
               # print('Forecast ACCURACY:----------')
               # print(forecast)
                # predict oof
                preds_idx = model_rfe.predict(X_te)
                #preds_idx=model_lgb.predict_proba(X_train.loc[test_index][features])[:,1]
                #print(len(preds_idx))
                #oof_preds.loc[test_index,'predictions'] = preds_idx
                oof_mse = metrics.mean_squared_error(y_te, preds_idx)
                oof_rmse = np.sqrt(metrics.mean_squared_error(y_te, preds_idx))
                print(f'Out of folds rmse of fold {i} is {oof_rmse}')
                print(f'Out of folds MSE of fold {i} is {oof_mse}')
                
                # predict test
                
                try:
                    x_test=x_test.drop(columns=cols_not_to_consider_modelling)
                except:
                    x_test=x_test
                
                
                preds += model_rfe.predict(x_test)
                #preds += model_rfe.predict(test_data[features]) / n_folds
    preds = np.round(preds / n_folds)
                #print('-'*50)
                #print('\n')
    return x_train,y_train,x_val,y_val,x_test,y_test,x_train_val,y_train_val,preds

In [75]:
#x_train,y_train,x_val,y_val,x_test,y_test,x_train_val,y_train_val=filter_data_scaled(model_data,2,cols_not_to_consider)

In [95]:
result = pd.DataFrame()
feature_importance=pd.DataFrame()
df_actuals_vs_forecast=pd.DataFrame()
for key,value in enumerate(UNIQUE_KEY):
    
    if True:#value==2.0 or value==46.0:
        
        
        val='BEST_PARAMS_'+str(value)
        with open(r'C:/Users/kishor.kukreja/OneDrive - Fractal Analytics Pvt. Ltd/Desktop/WIP/Tech Challenge/BestParams/'+val, 'rb') as f:
            globals()[f'best_rfe_params_{value}']=pickle.load(f)

        print('BEST PARAMETERS FOR:',value)
        print(globals()[f'best_rfe_params_{value}'])


        globals()[f'model_rfe_{value}'] = RandomForestRegressor(**globals()[f'best_rfe_params_{value}'],random_state=42,n_jobs=8)

        x_train,y_train,x_val,y_val,x_test,y_test,x_train_val,y_train_val,preds=fit_best_model_onesku_tscv(model_data,value,cols_not_to_consider,cols_not_to_consider_modelling,N_FOLDS,globals()[f'model_rfe_{value}'])
        
        
        
        
        ## Feature Importance
        sorted_idx = globals()[f'model_rfe_{value}'].feature_importances_.argsort()
        feature_imp = pd.DataFrame()
        feature_imp['feature'] = x_train_val.columns
        feature_imp['importance']  =  globals()[f'model_rfe_{value}'].feature_importances_
        feature_imp = feature_imp.sort_values(['importance'], ascending = False)
        feature_imp_disp = feature_imp.head(20)
        print(feature_imp_disp)
        sub_class=list(le_dict_PRODUCT_GROUP_CHANNEL_COUNTRY.keys())[list(le_dict_PRODUCT_GROUP_CHANNEL_COUNTRY.values()).index(value)]
        feature_imp_disp['Product group Channel Country']=sub_class
        feature_importance=feature_importance.append(feature_imp_disp)
        
        df_tmp=pd.DataFrame()
        df_tmp['actuals']=y_test
        df_tmp['forecast']=preds
        df_tmp['Product group Channel Country']=sub_class
        
        df_actuals_vs_forecast=df_actuals_vs_forecast.append(df_tmp)
        
        ##Saving Best Model
        val_MODEL='BEST_RFE_MODEL_'+str(value)
        with open(r'C:/Users/kishor.kukreja/OneDrive - Fractal Analytics Pvt. Ltd/Desktop/WIP/Tech Challenge/BestModel/'+val_MODEL, 'wb') as f:
            pickle.dump(globals()[f'model_rfe_{value}'], f)

        #print('Actuals',np.array(y_test))
        #print('Predictions',preds)
        # Creating a DataFrame from a Dictionary

        ##Per SKU 
        df = pd.DataFrame({'Product Group Counry Channel':value,'Week':np.array(x_test['WEEK']), 'actuals':np.array(y_test),
                  'predictions':preds})
        result = result.append(df)
        print('Overall MSE',mean_squared_error(df['actuals'],df['predictions']))
        print('Overall RMSE',rmse(df['actuals'],df['predictions']))
        print('Overall RMSLE',rmsle(df['actuals'],df['predictions']))
        #plt.rcParams["figure.figsize"] = (20,10)
        #sns.lineplot(x='Week',y= 'value', hue='variable',data=pd.melt(df, 'Week'))

BEST PARAMETERS FOR: 2.0
{'max_depth': 60, 'min_samples_leaf': 20, 'min_samples_split': 45, 'n_estimators': 600}
Training data Shape
(13450, 58)
(13450,)
Validation data Shape
(1345, 58)
(1345,)
Testing data Shape
(1345, 58)
(1345,)
Training+Validation data Shape
(14795, 58)
(14795,)
['WEEK_KEY', 'YEAR', 'ROLLING_WEEK']
(14795, 58)
Pass :- 5
Train period: 1 - 5 , Test period 6 - 55 # train records 1345 , # test records 13450
Pass :- 4
Train period: 1 - 15 , Test period 16 - 55 # train records 4035 , # test records 10760
Pass :- 3
Train period: 1 - 25 , Test period 26 - 55 # train records 6725 , # test records 8070
Pass :- 2
Train period: 1 - 35 , Test period 36 - 55 # train records 9415 , # test records 5380
Pass :- 1
Train period: 1 - 45 , Test period 46 - 55 # train records 12105 , # test records 2690
Out of folds rmse of fold 0 is 0.0345717296374692
Out of folds MSE of fold 0 is 0.001195204490126266
Out of folds rmse of fold 1 is 0.025674973412945012
Out of folds MSE of fold 1 is 0.

Out of folds rmse of fold 0 is 0.0013281255259997829
Out of folds MSE of fold 0 is 1.7639174128121998e-06
Out of folds rmse of fold 1 is 0.0012934510351091037
Out of folds MSE of fold 1 is 1.6730155802248117e-06
Out of folds rmse of fold 2 is 0.0008678399634936139
Out of folds MSE of fold 2 is 7.531462022365971e-07
Out of folds rmse of fold 3 is 0.0009914707243872346
Out of folds MSE of fold 3 is 9.830141973169478e-07
Out of folds rmse of fold 4 is 0.0012270208842699818
Out of folds MSE of fold 4 is 1.505580250434688e-06
                                feature  importance
16                   SALES_VOLUME_LAG_1    0.828028
43    EXPANDING_MEAN_SALES_VOLUME_4WEEK    0.069990
32     SALES_VOLUME_PRODUCT_GROUP_LAG_2    0.012841
1                                  WEEK    0.009771
34     SALES_VOLUME_PRODUCT_GROUP_LAG_4    0.006546
53                               KEY_LE    0.006251
30  SALES_VOLUME_PRODUCT_SUBGROUP_LAG_4    0.005373
35     SALES_VOLUME_PRODUCT_CLASS_LAG_1    0.005319
4    

Overall RMSLE 0.041162668074247205
BEST PARAMETERS FOR: 7.0
{'max_depth': 70, 'min_samples_leaf': 5, 'min_samples_split': 100, 'n_estimators': 700}
Training data Shape
(3750, 58)
(3750,)
Validation data Shape
(375, 58)
(375,)
Testing data Shape
(375, 58)
(375,)
Training+Validation data Shape
(4125, 58)
(4125,)
['WEEK_KEY', 'YEAR', 'ROLLING_WEEK']
(4125, 58)
Pass :- 5
Train period: 1 - 5 , Test period 6 - 55 # train records 375 , # test records 3750
Pass :- 4
Train period: 1 - 15 , Test period 16 - 55 # train records 1125 , # test records 3000
Pass :- 3
Train period: 1 - 25 , Test period 26 - 55 # train records 1875 , # test records 2250
Pass :- 2
Train period: 1 - 35 , Test period 36 - 55 # train records 2625 , # test records 1500
Pass :- 1
Train period: 1 - 45 , Test period 46 - 55 # train records 3375 , # test records 750
Out of folds rmse of fold 0 is 0.0023512719560507844
Out of folds MSE of fold 0 is 5.528479811310882e-06
Out of folds rmse of fold 1 is 0.001694633678028642
Out of 

KeyboardInterrupt: 

In [77]:
feature_importance.to_csv(r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Model Output\Feature Impportance.csv',index=False)

In [78]:
df_actuals_vs_forecast.to_csv(r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Model Output\ActualsForecast.csv',index=False)

In [79]:
#-------------- Forecast evaluation matrices

def rmsle_res(ts_sku):
    val = rmsle(ts_sku.predictions,ts_sku.actuals)
    return(val)
    
def mape_res(ts_sku):
    mape = np.mean(np.abs(ts_sku.predictions - ts_sku.actuals)/np.abs(ts_sku.actuals))  # MAPE
    return mape

def wmape(ts_sku):
    ts_sku = ts_sku.sort_values(by='Week',ascending=False).reset_index(drop=True)
    weights = np.arange(1,len(ts_sku)+1)
    #ts_sku =ts_sku.append(weights)
    no_weights = len(ts_sku)
    tot_weight = np.nansum(weights)
    ts_sku['wght'] = ((ts_sku.index+1)/tot_weight)
    ts_sku['ape'] = np.abs(ts_sku.predictions - ts_sku.actuals)/np.abs(ts_sku.actuals)+epsilon
    #print(np.nansum(ts_sku.wght))
    #print(ts_sku)
    
    wmape = np.mean((ts_sku.ape*ts_sku.wght))
    return wmape


#----------------command
r1=pd.DataFrame(result.groupby('Product Group Counry Channel').apply(rmsle_res).reset_index())
r2=pd.DataFrame(result.groupby('Product Group Counry Channel').apply(mape_res).reset_index())
r3=pd.DataFrame(result.groupby('Product Group Counry Channel').apply(wmape).reset_index())
r1.rename(columns = {0:'RMSLE'}, inplace = True)
r2.rename(columns = {0:'MAPE'}, inplace = True)
r3.rename(columns = {0:'WMAPE'}, inplace = True)

from functools import reduce
dfs= [r1,r2,r3]
df_final = reduce(lambda left, right: pd.merge(left,right, how = "left", on ='Product Group Counry Channel'), dfs)
df_final

,Product Group Counry Channel,RMSLE,MAPE,WMAPE
0,0.0,0.005663,1.0,0.000962
1,1.0,0.004639,1.0,0.000684
2,2.0,0.049600,1.0,0.000731
3,3.0,0.001519,1.0,0.023628
4,4.0,0.000756,1.0,0.106061
5,5.0,0.003647,1.0,0.040000
6,6.0,0.000437,1.0,0.133333
7,7.0,0.004021,1.0,0.002571
8,8.0,0.003193,1.0,0.002441
9,9.0,0.041163,1.0,0.001755


## Next 13 week forecast

In [82]:
def nextweek(x):
    ## 201609
    x=str(x)
    week=x[4:6]
    year=x[0:4]
    if week == '52':
        nextweek='01'
        nextyear=str(int(year)+1)
    else:
        nextweek=str(int(week)+1)
        nextyear=str(year)
    
    
    return nextyear+nextweek

In [85]:
all_sku_df = pd.DataFrame()

for key,value in enumerate(UNIQUE_KEY):
    
    sub_class=list(le_dict_PRODUCT_GROUP_CHANNEL_COUNTRY.keys())[list(le_dict_PRODUCT_GROUP_CHANNEL_COUNTRY.values()).index(value)]
    df_sub_class=df_data[(df_data['PRODUCT_GROUP_CHANNEL_COUNTRY']==sub_class)]
    
    df_pred=df_sub_class.copy()
    df_pred.rename(columns={'CURRENT_PRICE_X':'CURRENT_PRICE'},inplace=True)
    
    #---------------- No of SKU to Foreccast
    UNIQUE_SKU = df_sub_class.KEY.unique()
    print('Sub Class---------------------------------------------->',sub_class)
    
    if True:#value==2.0:
        for i,val in enumerate(UNIQUE_SKU):
            key=list(le_dict_KEY.keys())[list(le_dict_KEY.keys()).index(val)]
            print('Key---------------------------------------------->',key)
            sub_df = df_pred[df_pred.KEY==val]
            sub_df = sub_df[['PRODUCT_ID', 'CHANNEL', 'COUNTRY', 
                             'PRODUCT_SEASON', 'PRODUCT_GROUP',
                             'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS','PRODUCT_GROUP_CHANNEL_COUNTRY',
                             'KEY', 'ROLLING_WEEK','WEEK_KEY','WEEK','YEAR' ,'CURRENT_PRICE','ORIGINAL_PRICE', 'PREVIOUS_PRICE',
                            'SALES_VOLUME', ]]

            for j in range(1,FORECAST_HORIZON+1):

                #----------- Next Week addition
                max_week = np.max(sub_df['WEEK_KEY'])              
                roll_week = np.max(sub_df['ROLLING_WEEK'])+1
                #print('Maximum Week:',max_week)
                
                #print('roll_week',roll_week)

                next_week = nextweek(max_week)
                #print('Next Week:',next_week)
                #print(next_week)

                nextweek_df=  sub_df.groupby(['PRODUCT_ID', 'CHANNEL', 'COUNTRY','PRODUCT_SEASON', 'PRODUCT_GROUP',
                             'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS','PRODUCT_GROUP_CHANNEL_COUNTRY','KEY']).count().reset_index()
                nextweek_df = nextweek_df[['PRODUCT_ID', 'CHANNEL', 'COUNTRY','PRODUCT_SEASON', 'PRODUCT_GROUP',
                             'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS','PRODUCT_GROUP_CHANNEL_COUNTRY','KEY']]
                #print(nextweek_df)
                #nextweek_df['WEEK'] = str(next_week)[4:6]#
                #nextweek_df['YEAR'] = str(next_week)[0:4]
                nextweek_df['ROLLING_WEEK'] = roll_week
                nextweek_df['WEEK_KEY'] = next_week
                nextweek_df['WEEK'] = next_week[4:6]
                nextweek_df['YEAR'] = next_week[0:4]
                
                

                
                sub_df = pd.concat([sub_df,nextweek_df]).reset_index(drop=True)
                #print(sub_df)
                
                #--------------------- DATA Prep for Modelling
                sub_df['WEEK_KEY'] = sub_df['WEEK_KEY'].astype(int)
                sub_df['WEEK'] = sub_df['WEEK'].astype(int)
                sub_df['YEAR'] = sub_df['YEAR'].astype(int)

                #------ Feature Engineering------------------------------------------------
                df_sku=fe(sub_df)
                
                df_sku["CURRENT_PRICE"] = df_sku.groupby(['PRODUCT_ID', 'CHANNEL', 'COUNTRY'])['CURRENT_PRICE'].transform(lambda x: x.bfill())
                df_sku["CURRENT_PRICE"] = df_sku.groupby(['PRODUCT_ID', 'CHANNEL', 'COUNTRY'])['CURRENT_PRICE'].transform(lambda x: x.ffill())
                df_sku["CURRENT_PRICE"] = df_sku["CURRENT_PRICE"].fillna(0)


                df_sku["PREVIOUS_PRICE"] = df_sku.groupby(['PRODUCT_ID', 'CHANNEL', 'COUNTRY'])['PREVIOUS_PRICE'].transform(lambda x: x.bfill())
                df_sku["PREVIOUS_PRICE"] = df_sku.groupby(['PRODUCT_ID', 'CHANNEL', 'COUNTRY'])['PREVIOUS_PRICE'].transform(lambda x: x.ffill())
                df_sku["PREVIOUS_PRICE"] = df_sku["PREVIOUS_PRICE"].fillna(0)


                df_sku["ORIGINAL_PRICE"] = df_sku.groupby(['PRODUCT_ID', 'CHANNEL', 'COUNTRY'])['ORIGINAL_PRICE'].transform(lambda x: x.bfill())
                df_sku["ORIGINAL_PRICE"] = df_sku.groupby(['PRODUCT_ID', 'CHANNEL', 'COUNTRY'])['ORIGINAL_PRICE'].transform(lambda x: x.ffill())
                df_sku["ORIGINAL_PRICE"] = df_sku["ORIGINAL_PRICE"].fillna(0)
                
                df_sku["SALES_VOLUME"] = df_sku["SALES_VOLUME"].fillna(0)
                df_sku.SALES_VOLUME = df_sku.SALES_VOLUME.astype(float)
                
                


                df_sku[['START_DATE','END_DATE']]=df_sku.apply(lambda x : get_start_end_dates(x['YEAR'],x['WEEK']),axis=1,result_type="expand")
                df_sku['week_of_month']=df_sku.apply(lambda x: get_week_of_month(x['START_DATE']),axis=1)
                
                df_sku['is_month_start']=df_sku.apply(lambda x: 1 if x['week_of_month']==1 else 0,axis=1)
                df_sku['is_month_end']=df_sku.apply(lambda x: 1 if x['week_of_month']>=4 else 0,axis=1)
                df_sku['month']=df_sku['START_DATE'].dt.month
                df_sku['is_year_start']=df_sku.apply(lambda x: 1 if x['month']>=2 else 0,axis=1)
                df_sku['is_year_end']=df_sku.apply(lambda x: 1 if x['month']>=11 else 0,axis=1)
                df_sku['is_quarter_start']=df_sku.apply(lambda x: 1 if (x['month']==1 or x['month']==4 or x['month']==7 or x['month']==10 ) and x['week_of_month']==1 else 0,axis=1)
                df_sku['is_quarter_end']=df_sku.apply(lambda x: 1 if (x['month']==3 or x['month']==6 or x['month']==9 or x['month']==12 ) and x['week_of_month']>=1 else 0,axis=1)

                df_sku['week_sin365_1'] = np.sin(2*np.pi*df_sku['WEEK']/52)
                df_sku['week_cos365_1'] = np.cos(2*np.pi*df_sku['WEEK']/52)
                df_sku['week_sin365_2'] = np.sin(4*np.pi*df_sku['WEEK']/52)
                df_sku['week_cos365_2'] = np.cos(4*np.pi*df_sku['WEEK']/52)
                
                
                #df_sku['price_change_wrt_previous']=df_sku.apply(lambda x: ((x['CURRENT_PRICE']-x['PREVIOUS_PRICE'])/(x['PREVIOUS_PRICE']+epsilon))*100,axis=1)

                #df_sku['price_change_wrt_original']=df_sku.apply(lambda x: ((x['CURRENT_PRICE']-x['ORIGINAL_PRICE'])/(x['ORIGINAL_PRICE']+epsilon))*100,axis=1)
                #print(df_sku)

                df_sku=standard_column_names(df_sku)
                
                df_sku.fillna(0,inplace=True)
                
                unwanted_columns=['START_DATE', 'END_DATE', 
                                   'SALES_VOLUME_PRODUCT_SUBGROUP', 
                                   'SALES_VOLUME_PRODUCT_GROUP',
                                   'SALES_VOLUME_PRODUCT_CLASS', 
                                   'SALES_VOLUME_PRODUCT_SUBCLASS',
                                   'EXPANDING_MEAN_SALES_VOLUME',
                                   'EXPANDING_MAX_SALES_VOLUME',
                                   'EXPANDING_MIN_SALES_VOLUME']

                df_sku_cleaned=remove_columns(df_sku,unwanted_columns)

#                 ## Impute Nans to Zeros
                df_sku_cleaned.fillna(0,inplace=True)

                cat_features_no_one_hot = ['KEY','PRODUCT_ID', 'CHANNEL', 'COUNTRY','PRODUCT_SEASON', 'PRODUCT_GROUP',
                   'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS','PRODUCT_GROUP_CHANNEL_COUNTRY']
                
                for c in cat_features_no_one_hot:
                    df_sku_cleaned[c] = df_sku_cleaned[c].astype('category')

                id_cols=[]
                not_features = id_cols + list([LABEL])
                features = [cols for cols in df_sku_cleaned.columns if cols not in not_features]


                num_features=df_sku_cleaned.columns[df_sku_cleaned.dtypes!='category']
                num_features = num_features.drop(['ROLLING_WEEK',
                                 'IS_MONTH_START','IS_MONTH_END','IS_YEAR_START','IS_YEAR_END',
                                  'IS_QUARTER_START','IS_QUARTER_END',
                                  'WEEK_SIN365_1','WEEK_COS365_1', 'WEEK_SIN365_2', 'WEEK_COS365_2'])
                
                char_features=df_sku_cleaned.columns[df_sku_cleaned.dtypes=='category']
                for i, feature in enumerate(num_features):
                    df_sku_cleaned[feature].fillna(df_sku_cleaned[feature].median(), inplace = True)

#                 #--------------- label encoding
                data_sku_embedding =df_sku_cleaned.copy()

                for index,val in enumerate(cat_features):
                    new_col=val+'_LE'
                    data_sku_embedding[new_col]=globals()[f'label_encoder_{val}'].transform(data_sku_embedding[val].astype(str))
                    data_sku_embedding.drop(val,axis=1, inplace=True)
                    data_sku_embedding[new_col]=data_sku_embedding[new_col].astype('float')



                 #------------ Predition row--------------
                model_sku_data=data_sku_embedding.copy()
                model_sku_features=model_sku_data.columns.to_list()

                model_sku_features.remove(LABEL)
                model_sku_label=LABEL
                cols_sku_not_to_consider=['PRODUCT_ID_LE','WEEK_KEY','YEAR','ROLLING_WEEK']
                #cols_not_to_consider_modelling=['PRODUCT_ID_LE','WEEK_KEY','YEAR','ROLLING_WEEK']
                
                pred_row = model_sku_data[model_sku_data.ROLLING_WEEK==np.max(sub_df['ROLLING_WEEK'])]
                
                

                for index,column in enumerate(num_features):
                    new_col=column
                    #print(column)
                    #globals()[f'std_scalar_{col}'].fit(np.array(pred_row[col]).reshape(-1,1))
                    #with open(r'C:/Users/vikram.purohit/OneDrive - Fractal Analytics Pvt. Ltd/Documents/Projects/External/AES/Modelling/SS/'+column, 'rb') as f:
                        #globals()[f'std_scalar_{column}']=pickle.load(f)
                    pred_row[new_col]=globals()[f'min_max_scalar_{column}'].transform(np.array(pred_row[column]).reshape(-1,1))

                pred_sku_row = pred_row[model_sku_features]
                pred_sku_row.drop(columns=cols_sku_not_to_consider,inplace=True)
                #print(pred_sku_row)
                #print(pred_row)

                #---------------------------------- Call Best Model for Prediction----------------------------

                val_MODEL='BEST_RFE_MODEL_'+str(value)

                with open(r'C:/Users/kishor.kukreja/OneDrive - Fractal Analytics Pvt. Ltd/Desktop/WIP/Tech Challenge/BestModel/'+val_MODEL, 'rb') as f:
                    globals()[f'model_rfe_{value}']=pickle.load(f)

                predicted_value = globals()[f'model_rfe_{value}'].predict(pred_sku_row)

                #col = 'CONSUMPTION'
                #with open(r'C:/Users/vikram.purohit/OneDrive - Fractal Analytics Pvt. Ltd/Documents/Projects/External/AES/Modelling/SS/'+col, 'rb') as f:
                    #globals()[f'std_scalar_{col}']=pickle.load(f)
                predicted_value_transformed = globals()[f'min_max_scalar_{LABEL}'].inverse_transform([predicted_value])
                
                
                ## Decaying the forecats so as to not propagate the error
                
                predicted_value_transformed= predicted_value_transformed * np.exp(-0.05*i)
                predicted_value_transformed = np.round(predicted_value_transformed)

                #--------- Append new consumption---------------------------------------------------------
                #sub_df[sub_df.ROLLING_AES_WEEK==np.max(sub_df['ROLLING_AES_YEAR'])].CONSUMPTION = a

                sub_df.loc[sub_df.ROLLING_WEEK==np.max(sub_df['ROLLING_WEEK']), 'SALES_VOLUME'] = predicted_value_transformed
            #plt.plot(sub_df.SALES_VOLUME[0:60])
            #plt.plot(sub_df.SALES_VOLUME[60:])
            #plt.show()
            all_sku_df = pd.concat([all_sku_df,df_sku],axis=0)#all_sku_df.append(sub_df)

Sub Class----------------------------------------------> 26387251-B-STORES
Key----------------------------------------------> 135fc45eStoresB
Key----------------------------------------------> 25e07883StoresB
Key----------------------------------------------> 3cdb954fStoresB
Key----------------------------------------------> 4f6eec3cStoresB
Key----------------------------------------------> 5334cae3StoresB
Key----------------------------------------------> 59e5de32StoresB
Key----------------------------------------------> 651b3aecStoresB
Key----------------------------------------------> 661633eStoresB
Key----------------------------------------------> 7fc58fd9StoresB
Key----------------------------------------------> 8c66ba5eStoresB
Key----------------------------------------------> 8f22ae9aStoresB
Key----------------------------------------------> ba4fe7e1StoresB
Key----------------------------------------------> c52b07StoresB
Key----------------------------------------------> c9fa9e

Key----------------------------------------------> 9466d04fStoresB
Key----------------------------------------------> 29fea8c9StoresB
Key----------------------------------------------> 4bbe7575StoresB
Key----------------------------------------------> 6e4e484eStoresB
Key----------------------------------------------> 9cc6c58fStoresB
Key----------------------------------------------> ad795b92StoresB
Key----------------------------------------------> aff21c76StoresB
Key----------------------------------------------> d2d05b90StoresB
Key----------------------------------------------> f57697c2StoresB
Key----------------------------------------------> 13e01cb6StoresB
Key----------------------------------------------> 5ed28f61StoresB
Key----------------------------------------------> 69a6b89StoresB
Key----------------------------------------------> 159f75cfStoresB
Key----------------------------------------------> 699dd863StoresB
Key----------------------------------------------> 7030298cStor

Key----------------------------------------------> 289a1d88StoresB
Key----------------------------------------------> 886c3947StoresB
Key----------------------------------------------> 9497aa0eStoresB
Key----------------------------------------------> c727651StoresB
Key----------------------------------------------> 137e79c7StoresB
Key----------------------------------------------> 4995022fStoresB
Key----------------------------------------------> c458009aStoresB
Key----------------------------------------------> f92279fbStoresB
Key----------------------------------------------> fcc8c1b6StoresB
Key----------------------------------------------> 518ab92dStoresB
Key----------------------------------------------> 6d0ba44cStoresB
Key----------------------------------------------> a637dc81StoresB
Key----------------------------------------------> b882b4c7StoresB
Key----------------------------------------------> d66b2d5fStoresB
Key----------------------------------------------> de1454dbStor

Key----------------------------------------------> 76b0676StoresA
Key----------------------------------------------> 3ccfe56eStoresA
Key----------------------------------------------> ea319acbStoresA
Key----------------------------------------------> d263e33aStoresA
Key----------------------------------------------> da6b0ddfStoresA
Key----------------------------------------------> 1bc54294StoresA
Key----------------------------------------------> 3b0b680eStoresA
Key----------------------------------------------> 69092b8eStoresA
Key----------------------------------------------> b590f20bStoresA
Key----------------------------------------------> 6f2fb07cStoresA
Key----------------------------------------------> 898030ddStoresA
Key----------------------------------------------> 4669853StoresA
Key----------------------------------------------> 585e308cStoresA
Key----------------------------------------------> 68ddf21StoresA
Key----------------------------------------------> 714b755bStores

Key----------------------------------------------> 89359b26StoresB
Key----------------------------------------------> ef1ba93StoresB
Key----------------------------------------------> f4b7934StoresB
Key----------------------------------------------> 4ee1c288StoresB
Key----------------------------------------------> 59a9b4d8StoresB
Key----------------------------------------------> e3333919StoresB
Key----------------------------------------------> e4c5ea22StoresB
Key----------------------------------------------> 10516724StoresB
Key----------------------------------------------> d080e064StoresB
Key----------------------------------------------> 539d4438StoresB
Key----------------------------------------------> 4ac5fa43StoresB
Key----------------------------------------------> a07d1120StoresB
Key----------------------------------------------> bed34c81StoresB
Key----------------------------------------------> 1e3a269aStoresB
Key----------------------------------------------> 32d70cbfStore

...

Key----------------------------------------------> b594254fStoresA
Key----------------------------------------------> c4599b9bStoresA
Key----------------------------------------------> d3ecd298StoresA
Key----------------------------------------------> e0cede39StoresA
Key----------------------------------------------> e0fd100cStoresA
Key----------------------------------------------> e3b20625StoresA
Key----------------------------------------------> 3b610a31StoresA
Key----------------------------------------------> 4ca2dc23StoresA
Key----------------------------------------------> 5819590dStoresA
Key----------------------------------------------> 72a0074fStoresA
Key----------------------------------------------> 81b4e1fbStoresA
Key----------------------------------------------> 95e68521StoresA
Key----------------------------------------------> a79c5c41StoresA
Key----------------------------------------------> a81ed8faStoresA
Key----------------------------------------------> b3e4538fSto

Key----------------------------------------------> 80e33565OnlineB
Key----------------------------------------------> 815db1dbOnlineB
Key----------------------------------------------> 944fa4aaOnlineB
Key----------------------------------------------> f2ab51e0OnlineB
Key----------------------------------------------> 3338d518OnlineB
Key----------------------------------------------> 4cda07c3OnlineB
Key----------------------------------------------> 6419f64fOnlineB
Key----------------------------------------------> 688936c5OnlineB
Key----------------------------------------------> 6ed63b24OnlineB
Key----------------------------------------------> a34b76d5OnlineB
Key----------------------------------------------> adbfb590OnlineB
Key----------------------------------------------> bd1f872eOnlineB
Key----------------------------------------------> bdafb35fOnlineB
Key----------------------------------------------> 12ec7b5bOnlineB
Key----------------------------------------------> 5e926963Onl

Key----------------------------------------------> ffd13172OnlineB
Key----------------------------------------------> 139ca3f0OnlineB
Key----------------------------------------------> 21664ab4OnlineB
Key----------------------------------------------> 6843952cOnlineB
Key----------------------------------------------> 745f4ca3OnlineB
Key----------------------------------------------> 8873e358OnlineB
Key----------------------------------------------> e07dad55OnlineB
Key----------------------------------------------> 60e7611cOnlineB
Key----------------------------------------------> 98e1cfacOnlineB
Key----------------------------------------------> aaa7e9cfOnlineB
Key----------------------------------------------> 17e34865OnlineB
Key----------------------------------------------> 22ced95OnlineB
Key----------------------------------------------> 36311457OnlineB
Key----------------------------------------------> 5af9b741OnlineB
Key----------------------------------------------> 83743ab6Onli

Key----------------------------------------------> 78838610OnlineB
Key----------------------------------------------> ae5e3b78OnlineB
Key----------------------------------------------> be310af2OnlineB
Key----------------------------------------------> be63f01OnlineB
Key----------------------------------------------> e05995beOnlineB
Key----------------------------------------------> 22fa92e5OnlineB
Key----------------------------------------------> 37cad3a4OnlineB
Key----------------------------------------------> 3c3f7db5OnlineB
Key----------------------------------------------> 76786b3bOnlineB
Key----------------------------------------------> ea11be7aOnlineB
Key----------------------------------------------> 5345fa6bOnlineB
Key----------------------------------------------> 85db67faOnlineB
Key----------------------------------------------> 56f88d76OnlineB
Key----------------------------------------------> 91a18e49OnlineB
Key----------------------------------------------> 98de5742Onli

Key----------------------------------------------> d1159e63StoresA
Key----------------------------------------------> f6b832c7StoresA
Key----------------------------------------------> 58556e6dStoresA
Key----------------------------------------------> ef4fbb77StoresA
Key----------------------------------------------> 7d0e7bdaStoresA
Key----------------------------------------------> 5f20c88aStoresA
Key----------------------------------------------> 126d41d3StoresA
Key----------------------------------------------> 29a3c72cStoresA
Key----------------------------------------------> f951ace5StoresA
Key----------------------------------------------> cfe24bbbStoresA
Key----------------------------------------------> c642a43bStoresA
Key----------------------------------------------> bfadb5edStoresA
Key----------------------------------------------> d9bd3ebfStoresA
Key----------------------------------------------> 15018dcdStoresA
Key----------------------------------------------> 68263bb6Sto

In [86]:
all_sku_df.KEY.unique().shape

(1861,)

In [87]:
final_fore_data = all_sku_df.copy()
final_fore_data['A_F'] = np.where(final_fore_data['ROLLING_WEEK']>60,'Forecast','Actuals')
final_fore_data['WEEK'] = final_fore_data['WEEK_KEY'].astype(str).str[4:6].astype(int)
final_fore_data['YEAR'] = final_fore_data['WEEK_KEY'].astype(str).str[0:4].astype(int)
final_fore_data[['START_DATE','END_DATE']]=final_fore_data.apply(lambda x : get_start_end_dates(x['YEAR'],x['WEEK']),axis=1,result_type="expand")
final_fore_data['YEAR_QUARTER'] = final_fore_data['START_DATE'].dt.to_period('Q')
final_fore_data['YEAR_MONTH'] = final_fore_data['START_DATE'].apply(lambda x: x.strftime('%Y-%m'))
from datetime import date
    
final_fore_data['NO_DAYS_WEEK'] = (final_fore_data['END_DATE']-final_fore_data['START_DATE']).dt.days+1
final_fore_data['DAILY_VOLUME'] = final_fore_data.SALES_VOLUME/final_fore_data.NO_DAYS_WEEK
final_fore_data.shape

(135853, 74)

## Save Sales Forecast

In [88]:
final_fore_data.to_csv(r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Output\sales_volume_forecasts.csv',index=False)

In [89]:
final_fore_data[final_fore_data['KEY']=='82572fc9StoresB']

,PRODUCT_ID,CHANNEL,COUNTRY,PRODUCT_SEASON,PRODUCT_GROUP,PRODUCT_SUBGROUP,PRODUCT_CLASS,PRODUCT_SUBCLASS,PRODUCT_GROUP_CHANNEL_COUNTRY,KEY,ROLLING_WEEK,WEEK_KEY,WEEK,YEAR,CURRENT_PRICE,ORIGINAL_PRICE,PREVIOUS_PRICE,SALES_VOLUME,SALES_VOLUME_LAG_1,SALES_VOLUME_LAG_2,SALES_VOLUME_LAG_3,SALES_VOLUME_LAG_4,SALES_VOLUME_LAG_5,SALES_VOLUME_LAG_6,SALES_VOLUME_LAG_7,SALES_VOLUME_LAG_8,SALES_VOLUME_LAG_9,SALES_VOLUME_LAG_10,SALES_VOLUME_LAG_11,SALES_VOLUME_PRODUCT_SUBGROUP,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_1,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_2,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_3,SALES_VOLUME_PRODUCT_SUBGROUP_LAG_4,SALES_VOLUME_PRODUCT_GROUP,SALES_VOLUME_PRODUCT_GROUP_LAG_1,SALES_VOLUME_PRODUCT_GROUP_LAG_2,SALES_VOLUME_PRODUCT_GROUP_LAG_3,SALES_VOLUME_PRODUCT_GROUP_LAG_4,SALES_VOLUME_PRODUCT_CLASS,SALES_VOLUME_PRODUCT_CLASS_LAG_1,SALES_VOLUME_PRODUCT_CLASS_LAG_2,SALES_VOLUME_PRODUCT_CLASS_LAG_3,SALES_VOLUME_PRODUCT_CLASS_LAG_4,SALES_VOLUME_PRODUCT_SUBCLASS,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_1,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_2,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_3,SALES_VOLUME_PRODUCT_SUBCLASS_LAG_4,EXPANDING_MEAN_SALES_VOLUME_4WEEK,EXPANDING_MEAN_SALES_VOLUME_12WEEK,EXPANDING_MEAN_SALES_VOLUME_8WEEK,EXPANDING_MIN_SALES_VOLUME,EXPANDING_MAX_SALES_VOLUME,EXPANDING_MEAN_SALES_VOLUME,START_DATE,END_DATE,WEEK_OF_MONTH,IS_MONTH_START,IS_MONTH_END,MONTH,IS_YEAR_START,IS_YEAR_END,IS_QUARTER_START,IS_QUARTER_END,WEEK_SIN365_1,WEEK_COS365_1,WEEK_SIN365_2,WEEK_COS365_2,A_F,YEAR_QUARTER,YEAR_MONTH,NO_DAYS_WEEK,DAILY_VOLUME
0,82572fc9,Stores,B,L,606565a1,9d3d7fe0,aea70009,7b771b3e,606565a1-B-STORES,82572fc9StoresB,1,201502,2,2015,14.93,14.93,14.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,2015-01-05,2015-01-11,2,0,0,1,0,0,0,0,2.393157e-01,9.709418e-01,4.647232e-01,0.885456,Actuals,2015Q1,2015-01,7,0.000000
1,82572fc9,Stores,B,L,606565a1,9d3d7fe0,aea70009,7b771b3e,606565a1-B-STORES,82572fc9StoresB,2,201503,3,2015,14.93,14.93,14.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,2015-01-12,2015-01-18,3,0,0,1,0,0,0,0,3.546049e-01,9.350162e-01,6.631227e-01,0.748511,Actuals,2015Q1,2015-01,7,0.000000
2,82572fc9,Stores,B,L,606565a1,9d3d7fe0,aea70009,7b771b3e,606565a1-B-STORES,82572fc9StoresB,3,201504,4,2015,14.93,14.93,14.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,2015-01-19,2015-01-25,4,0,1,1,0,0,0,0,4.647232e-01,8.854560e-01,8.229839e-01,0.568065,Actuals,2015Q1,2015-01,7,0.000000
3,82572fc9,Stores,B,L,606565a1,9d3d7fe0,aea70009,7b771b3e,606565a1-B-STORES,82572fc9StoresB,4,201505,5,2015,14.93,14.93,14.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,2015-01-26,2015-02-01,5,0,1,1,0,0,0,0,5.680647e-01,8.229839e-01,9.350162e-01,0.354605,Actuals,2015Q1,2015-01,7,0.000000
4,82572fc9,Stores,B,L,606565a1,9d3d7fe0,aea70009,7b771b3e,606565a1-B-STORES,82572fc9StoresB,5,201506,6,2015,14.93,14.93,14.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,2015-02-02,2015-02-08,1,1,0,2,1,0,0,0,6.631227e-01,7.485107e-01,9.927089e-01,0.120537,Actuals,2015Q1,2015-02,7,0.000000
5,82572fc9,Stores,B,L,606565a1,9d3d7fe0,aea70009,7b771b3e,606565a1-B-STORES,82572fc9StoresB,6,201507,7,2015,14.93,14.93,14.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,2015-02-09,2015-02-15,1,1,0,2,1,0,0,0,7.485107e-01,6.631227e-01,9.927089e-01,-0.120537,Actuals,2015Q1,2015-02,7,0.00

In [90]:

#82572fc9StoresB
#38bc251fStoresB